In [23]:
import pandas as pd
import json

# チームIDをチーム名に変換する辞書作成
team_df = pd.read_csv('/Users/onehe/Desktop/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/team.csv')
team_dic = dict(zip(team_df['team_id'], team_df['team_name']))


# ○: フル出場, ▲: 途中出場, ▽: 途中退場, ◆: 途中出場途中退場, ×: 出場停止, ※: 他大会の影響で出場停止, B: ベンチ入り
# 各々の回数をカウント
tmp = ['rat_full_play', 'rat_out_play', 'rat_in_play', 'rat_inout_play', 'rat_bench_play', 'rat_susp_play']
colnames = []
colnames += [i + '_first' for i in tmp]
colnames += [i + '_second' for i in tmp]
characters = ['○', '▲', '▽', '◆', 'B', '×'] * 2

df["out_of_bench_first"] = 0
df["out_of_bench_second"] = 0

# 2015 ~ 2018シーズンのjsonをcsvに変換
for year in range(2015, 2019):
    dfs = []
    team_df_year = team_df[team_df.year == year]
    for div, team_id in zip(team_df_year["div"], team_df_year["team_id"]):
        # 2stageかどうか
        is_2stage = year < 2017 and div == "J1"

        # 後で１試合あたりの指標に変換するために試合数の情報を記憶
        if div == 'J1':
            num_match = 34
        else:
            num_match = 42

        if is_2stage:
            with open(f"/Users/onehe/Desktop/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}_s1.json", encoding="utf-8_sig") as f:
                first_dic = json.load(f)
            with open(f"/Users/onehe/Desktop/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}_s2.json", encoding="utf-8_sig") as f:
                second_dic = json.load(f)

            first_df = pd.DataFrame.from_dict(first_dic, orient='index')
            second_df = pd.DataFrame.from_dict(second_dic, columns=list(range(17, 34)), orient='index')
            #(first_df)
            full_df = pd.concat([first_df, second_df], axis=1, join='outer')
            first_df = full_df.iloc[:, :num_match // 2]
            #print(first_df)
            second_df = full_df.iloc[:, num_match // 2:]
        else:
            with open(f"/Users/onehe/Desktop/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}.json", encoding="utf-8_sig") as f:
                dic = json.load(f)

            full_df = pd.DataFrame.from_dict(dic, orient='index')

            # シーズン前半・後半に分けて集計するためにdataframeを分割
            first_df = full_df.iloc[:, :num_match // 2]
            second_df = full_df.iloc[:, num_match // 2:]

        data = []
        for colname, ch in zip(colnames, characters):
            if 'first' in colname:
                data.append(first_df.apply(lambda d: d.str.contains(ch)).sum(axis=1).values)
            elif 'second' in colname:
                data.append(second_df.apply(lambda d: d.str.contains(ch)).sum(axis=1).values)
            else:
                data.append(full_df.apply(lambda d: d.str.contains(ch)).sum(axis=1).values)
        # 1試合あたりの値に変換
        df = pd.DataFrame(data).T
        df /= num_match
        df.columns = colnames
        df['team_id'] = team_id
        df['team'] = df['team_id'].map(int).map(team_dic)
        df.drop(['team_id'], axis=1, inplace=True)
        df['name'] = full_df.index
        #df["out_of_bench"] = 0
        #df["out_of_bench"] = 1 - df['rat_full_play'] - df['rat_out_play'] - df['rat_in_play'] - df['rat_inout_play'] - df['rat_bench_play'] - df['rat_susp_play']

        dfs.append(df)

    df = pd.concat(dfs)
    df.to_csv(f'/Users/onehe/Desktop/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/event_play_{year}改.csv', index=False)

In [19]:
df

,rat_full_play_first,rat_out_play_first,rat_in_play_first,rat_inout_play_first,rat_bench_play_first,rat_susp_play_first,out_of_bench_first,rat_full_play_second,rat_out_play_second,rat_in_play_second,rat_inout_play_second,rat_bench_play_second,rat_susp_play_second,out_of_bench_second,team,name
0,0.441176,0.000000,0.000000,0.0,0.058824,0.000000,0.5,0.352941,0.000000,0.000000,0.0,0.000000,0.00000,0.5,鹿島,クォン スンテ
1,0.029412,0.029412,0.147059,0.0,0.058824,0.000000,0.5,0.000000,0.117647,0.029412,0.0,0.147059,0.00000,0.5,鹿島,内田 篤人
2,0.294118,0.000000,0.058824,0.0,0.058824,0.029412,0.5,0.117647,0.000000,0.000000,0.0,0.029412,0.00000,0.5,鹿島,昌子 源
3,0.147059,0.058824,0.029412,0.0,0.117647,0.000000,0.5,0.323529,0.029412,0.000000,0.0,0.029412,0.00000,0.5,鹿島,レオ シルバ
4,0.411765,0.000000,0.000000,0.0,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.5,鹿島,植田 直通
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,0.000000,0.023810,0.000000,0.0,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.5,水戸,永坂 勇人
31,0.023810,0.214286,0.095238,0.0,0.047619,0.000000,0.5,0.071429,0.119048,0.285714,0.0,0.000000,0.00000,0.5,水戸,伊藤 涼太郎
32,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.5,0.000000,0.214286,0.071429,0.0,0.023810,0.02381,0.5,水戸,バティスタ
33,0.000000,0.119048,0.047619,0.0,0.190476,0.000000,0.5,0.000000,0.047619,0.000000,0.0,0.023810,0.00000,0.5,水戸,齋藤 恵太
